# Master course in Object Recognition
## Practice 1

### Title: Deep learning advanced architectures

The goal is to practice advanced deep learning architectures for multi-label classification in [Pascal VOC dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/index.html). We specifically check ResNet50, Inception and MobileNet. We will see 1) how pretrained ResNet50 on imagenet performs on multi-label images, 2) how to modify classification head and 3) implementation of F1 metric.

### NOTES

- Hyperparameters are modifiable,
- The dataset is PASCAL VOC 2012,
- The code uses the KERAS library,
- The code can run in google colab.
- How to finetune on a pretrained model not included (i.e. freeze the pretrained network and train the head, then finetune everything),
- No validation set has been defined. The test and validation sets are the same.

In [1]:
# Imports

import pandas as pd
import time

In [2]:
# Importing from .py files

from config import *
from experiment_config import experiments
from train_and_test import train_and_test
from load_data import load_data, create_dataset
from models import create_model

In [3]:
train_list = load_data(TRAIN_TXT)
test_list = load_data(TEST_TXT)

# Create dictionaries to store datasets for different batch sizes
train_datasets = {}
test_datasets = {}

start_time = time.time()
# Iterate over batch sizes and create datasets
for batch_size in BATCH_SIZES:
    train_datasets[batch_size] = create_dataset(
        train_list, batch_size, is_training=True
    )
    test_datasets[batch_size] = create_dataset(test_list, batch_size, is_training=False)
print(f"Time taken to create datasets: {time.time() - start_time} seconds")

c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\.venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(



Time taken to create datasets: 3.123579263687134 seconds


In [5]:
# Run model experiments
exp_name = "model-experiments"
for exp in experiments[exp_name]:
    
    # Create the model
    base_model, model = create_model(exp, exp_name)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, base_model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

Defining model: resnet50 no-pretraining no-warmup
Starting training: resnet50 no-pretraining no-warmup
Time taken for training one epoch: 20.20s
Epoch 0 - Train Loss: 0.7207, Acc: 0.5696, F1: 0.1080, mAP: 0.3620
Time taken for testing one epoch: 4.94s
Epoch 0 - Test Loss: 0.6381, Acc: 0.5680, F1: 0.2367, mAP: 0.3312
Results saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\02_results\model-experiments.csv
Model saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\models/resnet50-0.weights.h5
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/resnet50-0-train_loss.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/resnet50-0-train_acc.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practical

c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\src\models.py:78: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mynet(include_top=False)


Starting training: mobilenet_v2 no-pretraining no-warmup
Time taken for training one epoch: 9.09s
Epoch 0 - Train Loss: 0.6471, Acc: 0.4850, F1: 0.0833, mAP: 0.3185
Time taken for testing one epoch: 2.05s
Epoch 0 - Test Loss: 0.7657, Acc: 0.6230, F1: 0.2367, mAP: 0.3668
Results saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\02_results\model-experiments.csv
Model saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\models/mobilenet_v2-6.weights.h5
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/mobilenet_v2-6-train_loss.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/mobilenet_v2-6-train_acc.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/mobilenet_v2

c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\src\models.py:80: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  else mynet(weights="imagenet", include_top=False)


Starting training: mobilenet_v2 pretraining no-warmup
Time taken for training one epoch: 8.59s
Epoch 0 - Train Loss: 0.6093, Acc: 0.4900, F1: 0.1833, mAP: 0.2328
Time taken for testing one epoch: 2.00s
Epoch 0 - Test Loss: 0.7182, Acc: 0.6293, F1: 0.2367, mAP: 0.4154
Results saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\02_results\model-experiments.csv
Model saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\models/mobilenet_v2-7.weights.h5
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/mobilenet_v2-7-train_loss.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/mobilenet_v2-7-train_acc.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/mobilenet_v2-7-

In [6]:
# Determine the best experiment of the 9 model experiments

df = pd.read_csv(RESULTS_DIR / f"model-experiments.csv")
best_id = df.loc[df["Test mAP"].idxmax(), "ID"]

best_model_experiment_config = next(
    exp for exp in experiments["model-experiments"] if exp.id == best_id
)

best_model_experiment_config

ExperimentConfig(id=7, title='mobilenet_v2 pretraining no-warmup', net_name=['mobilenet_v2', 'MobileNetV2'], train_from_scratch=False, warm_up=False, batch_size=32, n_epochs=1, last_layer_activation='sigmoid', learning_rate=0.001, loss='binary_crossentropy', classifier_head='default')

In [7]:
# Run hyperparameter experiments

exp_name = "hyperparameter-experiments"
for exp in experiments[exp_name]:

    # Create the model
    base_model, model = create_model(exp, exp_name, best_model_experiment_config)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, base_model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

Defining model: batch_size: 16, learning_rate: 0.001
Reusing net_name: ['mobilenet_v2', 'MobileNetV2'], train_from_scratch: False, warm_up: False from best model experiment


c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\src\models.py:80: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  else mynet(weights="imagenet", include_top=False)


Starting training: batch_size: 16, learning_rate: 0.001
Time taken for training one epoch: 6.02s
Epoch 0 - Train Loss: 0.6093, Acc: 0.5790, F1: 0.1523, mAP: 0.3206


c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\.venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Time taken for testing one epoch: 1.55s
Epoch 0 - Test Loss: 0.5424, Acc: 0.5062, F1: 0.0667, mAP: 0.2226
Results saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\02_results\hyperparameter-experiments.csv
Model saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\models/mobilenet_v2-9.weights.h5
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/mobilenet_v2-9-train_loss.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/mobilenet_v2-9-train_acc.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/mobilenet_v2-9-train_f1.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/mobilenet_v2-9-

In [4]:
# Determine the best experiment of the 9 hyperparameter experiments

df = pd.read_csv(RESULTS_DIR / f"hyperparameter-experiments.csv")
best_id = df.loc[df["Test mAP"].idxmax(), "ID"]

best_hyperparameter_experiment_config = next(
    exp for exp in experiments["hyperparameter-experiments"] if exp.id == best_id
)

best_hyperparameter_experiment_config

ExperimentConfig(id=15, title='batch_size: 64, learning_rate: 0.001', net_name=['resnet50', 'ResNet50'], train_from_scratch=False, warm_up=True, batch_size=64, n_epochs=1, last_layer_activation='sigmoid', learning_rate=0.001, loss='binary_crossentropy', classifier_head='default')

In [5]:
# Run augmentation experiments

exp_name = "augmentation-experiments"
for exp in experiments[exp_name]:

    # Create the model
    base_model, model = create_model(exp, exp_name, best_hyperparameter_experiment_config)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, base_model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

In [5]:
# Run classifier head experiments

exp_name = "classfier_head-experiments"
for exp in experiments[exp_name]:

    # Create the model
    base_model, model = create_model(exp, exp_name, best_hyperparameter_experiment_config)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, base_model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

Defining model: classifier_head: ensemble
Reusing net_name: ['resnet50', 'ResNet50'], train_from_scratch: False, warm_up: True from best hyperparameter experiment
Starting training: classifier_head: ensemble
Freezing base model layers...
Time taken for training one epoch: 19.94s
Epoch 0 - Train Loss: 0.7702, Acc: 0.4769, F1: 0.1211, mAP: 0.1893


c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\.venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Time taken for testing one epoch: 8.88s
Epoch 0 - Test Loss: 0.6247, Acc: 0.5829, F1: 0.1818, mAP: 0.2600
Results saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\02_results\classfier_head-experiments.csv
Model saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\models/resnet50-19.weights.h5
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/resnet50-19-train_loss.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/resnet50-19-train_acc.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/resnet50-19-train_f1.csv
History saved to c:\Users\sanch\Desktop\MAI\Semester2\OR\Deliverables\mai-object-recognition\practicals\p1\data\01_histories/resnet50-19-train_map.csv
H